# Designing a workflow agent pattern

## Objective

To build a robust "Task-Optimized Solver" that maximizes answer quality by combining three Agentic Patterns: Parallelization, Evaluation, and Routing. The goal is to first generate the highest-quality problem possible through competition, and then dynamically assign that problem to the specific AI model best suited to solve it.

**Workflow Steps**

Phase 1: Generation (Parallelization)

- Fan-Out: Simultaneously prompt three different models to generate a challenging problem (math, reasoning, or coding).

- Aggregator: Collect the three candidate problems into a single list.

- Judge: Use a model (e.g., GPT-4o) to evaluate the candidates and output only the single best problem.

Phase 2: Classification (Evaluator). 

- Analysis: Classify the question into a topic.

- Decision: The evaluator classifies the problem into one of three domains: MATH, REASONING, or CODE.

Phase 3: Specialized Execution (Router).

- Routing: Automatically direct the problem to the domain specialist: * Math: Send to specific model. * Reasoning: Send to another. * Code: Send to another.

- Final Output: Display the specialized solution.

## Set up the enviroment

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)

In [ ]:
# Set up the API key variables

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

## Phase 1: Generation (Parallelization)

In [ ]:
# Define the request

request = "Please design a single, difficult question to evaluate the specifyc intelligence of an LLM."
request += "You must independently choose to make it one of these three types:"
request += "1) A Mathematical Problem, 2) A logical Reasoning Puzzle, or 3) A coding Challenge."
request += "Do not tell me wich category you chose. Just output the question itself"

# Create the messages object the we will send to all competitors
messages = [{
    "role": "user",
    "content": request
}]

print(messages)

In [ ]:
# Initialize lists
competitors = []
questions = []

In [ ]:
# Gemini call
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash-preview-09-2025"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
questions.append(answer)

In [ ]:
# Groq call
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"

response = groq.chat.completions.create(
    model=model_name, 
    messages=messages
    )
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
questions.append(answer)

In [ ]:
# Ollama call
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(
    model=model_name, 
    messages=messages
    )
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
questions.append(answer)

In [ ]:
print(competitors)
print(questions)

In [ ]:
# prepare the data for the judge 
# we zip the lists together

for competitor, question in zip(competitors, questions):
    print(f"Competitor: {competitor}\n\n{question}")

In [ ]:
# Aggregation building

# We define a string variable to hold all the options
candidates_questions = ""

# We use enumerate to assign IDs automatically
for index, question in enumerate(questions):
    # index is 0, then 1, then 2 ...
    # answer is the actual text
    # we use {index+a} because humans count from 1, but Python counts from 0
    candidates_questions += f"# Question proposed from competitor {index+1}\n\n"
    candidates_questions += question + "\n\n"


In [ ]:
print(candidates_questions)

In [ ]:
# We define the judge´s system prompt
judge = f"""
You are an expert in IQ detection level.
Here are 3 candidates question generated by AI models to evaluate their intelligence:

{candidates_questions}

Task: Analyze these questions. Select the SINGLE most accurate question to measure the intelligence level of an LLM.

Return your decision in this exact JSON format, response ONLY in JSON:
{{
    "winner_id": <number 1, 2 or 3>,
    "reason": "<short explanation in PLAIN ENGLISH ONLY. Do not use LaTeX, backslashes, or markdown>"
}}
"""
print(judge)

In [ ]:
# Judge messages
judge_messages = [{
    "role":"user",
    "content": judge
    }]

print(judge_messages)

In [ ]:
# Call the judge

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash-preview-09-2025"

response = gemini.chat.completions.create(model=model_name, messages=judge_messages)
winner_text = response.choices[0].message.content

print(winner_text)

In [ ]:
# Clean the output
winner_text = winner_text.replace("```json", "").replace("```", "").strip()

try:
    # Parse the JSON 
    winner_dict = json.loads(winner_text)
    
    # Get the ID
    winning_id = int(winner_dict["winner_id"])
    
    # RETRIEVE THE QUESTION FROM YOUR LIST
    selected_question = questions[winning_id - 1]
    
    print(f"Winner ID: {winning_id}")
    print(f"Reason: {winner_dict['reason']}")
    print("-" * 50)
    print(f"Selected Question (Safe Retrieve):\n{selected_question[:200]}...")

except json.JSONDecodeError as e:
    print(f"JSON Error: {e}")
    print(f"Raw Output: {winner_text}")

In [ ]:
selected_question

## Phase 2: Classification (Evaluator)

In [ ]:
print(selected_question)

In [ ]:
# Define the Router Prompt

router_prompt = f"""
You are an intelligent classifier agent.
Classify the following question into exactly one of these three categories:
- MATH
- CODE
- Reasoning

Question:
"{selected_question}"

Output ONLY the category name. Do not explain.
"""

In [ ]:
# Call the Router (using Gemini for free)

# Router messages
router_messages = [{
    "role":"user",
    "content": router_prompt
    }]

print(router_messages)


In [ ]:
# Call the router

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash-preview-09-2025"

response = gemini.chat.completions.create(
    model=model_name, 
    messages=router_messages
    )

topic = response.choices[0].message.content

print(f"Router decision: {topic}")

## Phase 3: Specialized Execution (Router)

In [ ]:
# We use an IF/ELSE block to direct traffic base on the "topic" variables

if "MATH" in topic:
    print("Routing to math expert LLM ...")

    # Create a specific prompt for the Math Expert
    math_prompt = f"You are a mathematician. Solve this problem step-by-step, showing all work:\n\n{selected_question}"

    math_messages = [{
        "role":"user",
        "content": math_prompt
        }]

    gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    model_name = "gemini-2.5-flash-lite"

    response = gemini.chat.completions.create(
        model= model_name, 
        messages= math_messages
        )

    math_response = response.choices[0].message.content

    display(Markdown(selected_question))
    display(Markdown(math_response))

elif "CODE" in topic:
    print("Routing to code expert LLM ...")

    # Create a specific prompt for the CodeExpert
    code_prompt = f"You are a Senior Python Developer. Write efficient code to solve this:\n\n{selected_question}"

    code_messages = [{
        "role":"user",
        "content": code_prompt
        }]

    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    model_name = "llama3.2"

    response = ollama.chat.completions.create(
        model= model_name, 
        messages= code_messages
        )

    code_response = response.choices[0].message.content

    display(Markdown(selected_question))
    display(Markdown(code_response))

elif "REASONING" in topic:
    print("Routing to reasoning expert LLM ...")

    # Create a specific prompt for the Reasoning Expert
    reasoning_prompt = f"Solve this logic puzzle clearly:\n\n{selected_question}"

    reasoning_messages = [{
        "role":"user",
        "content": reasoning_prompt
        }]

    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    model_name = "llama3.2"

    response = ollama.chat.completions.create(
        model= model_name, 
        messages= reasoning_messages
        )

    reasoning_response = response.choices[0].message.content

    display(Markdown(selected_question))
    display(Markdown(reasoning_response))

else:
    print(f"Error: Router returned unknow topic '{topic}'")
